## Siemens DICOM Physiological Data
Read physio data from a CMRR Multiband generated DICOM file (DICOM spectroscopy format)  
Current CMRR MB sequence version : VE11C R016a  
Author : Mike Tyszka  
Place  : Caltech Brain Imaging Center  
Dates  : 2018-03-29 JMT From scratch  

In [92]:
import numpy as np
import pydicom
from scipy.interpolate import interp1d
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [93]:
# Create DICOM object
d = pydicom.read_file('physio.dcm')

# Extract data from Siemens spectroscopy tag (0x7fe1, 0x1010)
# Convert from a bytes literal to a UTF-8 encoded string, ignoring errors
physio_string = d[0x7fe1, 0x1010].value.decode('utf-8', 'ignore')

# CMRR DICOM physio log has \n separated lines
physio_lines = physio_string.splitlines()

In [2]:
print(physio_lines[1:10])

['ScanDate    = 20180326_152027', 'LogVersion  = EJA_1', 'LogDataType = PULS', 'SampleTime  = 2', '', 'ACQ_TIME_TICS  CHANNEL  VALUE  SIGNAL', '', '     22103303     PULS   2926 ', '     22103305     PULS   2969 ']


### Parse the pulse and respiratory waveforms to a numpy array

In [21]:
t_puls = []
t_resp = []
s_puls = []
s_resp = []

current_waveform = ''

for line in physio_lines:
    
    parts = line.split()
    
    if len(parts) > 2:
        
        if 'ScanDate' in parts[0]:
            scan_date = parts[2]
            
        if 'LogDataType' in parts[0]:
            current_waveform = parts[2]
            
        if 'SampleTime' in parts[0]:
            
            if 'PULS' in current_waveform:
                dt_puls = float(parts[2]) * 1e-3

            if 'RESP' in current_waveform:
                dt_resp = float(parts[2]) * 1e-3
                
        if 'PULS' in parts[1]:
            t_puls.append(float(parts[0]))
            s_puls.append(float(parts[2]))
            
        if 'RESP' in parts[1]:
            t_resp.append(float(parts[0]))
            s_resp.append(float(parts[2]))
            
# Convert to numpy arrays
t_puls = np.array(t_puls)
t_resp = np.array(t_resp)
s_puls = np.array(s_puls)
s_resp = np.array(s_resp)

# Zero time origin (identical offset for all waveforms) and scale to seconds
t_puls = (t_puls - t_puls[0]) * dt_puls
t_resp = (t_resp - t_resp[0]) * dt_resp

### Resample respiration waveform to match pulse waveform timing

In [24]:
f = interp1d(t_resp, s_resp, kind='cubic', fill_value='extrapolate')
s_resp_i = f(t_puls)

### Plot physiological waveforms

In [48]:
output_notebook()

p = figure(title='Scan Date : '+scan_date, plot_width=1000, plot_height=400)

idx = slice(0, 10000, 10)
t_short = t_puls[idx]
s_puls_short = s_puls[idx]
s_resp_short = s_resp_i[idx]

p.line(t_short, s_puls_short, color='red')
p.line(t_short, s_resp_short, color='blue', line_width=2)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Waveform Amplitude (AU)'

show(p)

Loading BokehJS ...

### Respiratory waveform distribution

In [91]:
p = figure(title='Scan Date : '+scan_date, plot_width=1000, plot_height=400)

hist, edges = np.histogram(s_resp, density=True, bins=100)
centers = (edges[:-1] + edges[1:])/2.0

hist[hist > 1e-3] = 1e-3

p.vbar(centers, width=30.0, top=hist)
p.xaxis.axis_label = 'Respiratory waveform signal'
show(p)

In [34]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

p = figure(plot_width=800, plot_height=400)

p.line(t, puls)

show(p)

Loading BokehJS ...